In [1]:
# -------------------------------------------------------------
# 1. Imports and Setup
# -------------------------------------------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

# Select device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Using device: {DEVICE}")

# -------------------------------------------------------------
# 2. ResNet50 Model Definition
# -------------------------------------------------------------
class ResNetModel(nn.Module):
    def __init__(self, num_classes=2, pretrained=True):
        super(ResNetModel, self).__init__()
        if pretrained:
            weights = models.ResNet50_Weights.DEFAULT
        else:
            weights = None

        self.resnet = models.resnet50(weights=weights)

        # Optional: freeze early convolution layers
        for param in self.resnet.layer1.parameters():
            param.requires_grad = False
        for param in self.resnet.layer2.parameters():
            param.requires_grad = False

        # Replace final fully connected layer
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# -------------------------------------------------------------
# 3. Load Dataset from Data Folder
# -------------------------------------------------------------
DATA_ROOT = r"C:\Users\Admin\OneDrive\Desktop\ResearchTrack\CASIA_FL_Project\data"

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet input size
    transforms.ToTensor(),
])

# Expect folder structure: data/Authentic, data/Tampered
full_dataset = datasets.ImageFolder(DATA_ROOT, transform=transform)

# Split dataset: 75% train, 20% test, 5% eval
train_size = int(0.75 * len(full_dataset))
test_size = int(0.20 * len(full_dataset))
eval_size = len(full_dataset) - train_size - test_size
train_set, test_set, eval_set = random_split(full_dataset, [train_size, test_size, eval_size])

print(f"✅ Dataset loaded from: {DATA_ROOT}")
print(f"Total images: {len(full_dataset)}")
print(f"Train: {len(train_set)} | Test: {len(test_set)} | Eval: {len(eval_set)}")

# -------------------------------------------------------------
# 4. Evaluation Function
# -------------------------------------------------------------
def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    return acc, precision, recall, f1

# -------------------------------------------------------------
# 5. Centralized Training (Single Model)
# -------------------------------------------------------------
model = ResNetModel(num_classes=2, pretrained=True).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

trainloader = DataLoader(train_set, batch_size=32, shuffle=True)
testloader = DataLoader(test_set, batch_size=32, shuffle=False)

EPOCHS = 15
print("\n🚀 Starting centralized training...")

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for images, labels in trainloader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(trainloader)
    acc, precision, recall, f1 = evaluate_model(model, testloader)

    print(f"\n📊 Epoch [{epoch+1}/{EPOCHS}]")
    print(f"Loss      : {avg_loss:.4f}")
    print(f"Accuracy  : {acc:.4f}")
    print(f"Precision : {precision:.4f}")
    print(f"Recall    : {recall:.4f}")
    print(f"F1 Score  : {f1:.4f}")
    print("-" * 60)

# -------------------------------------------------------------
# 6. Save Trained Model
# -------------------------------------------------------------
torch.save(model.state_dict(), "resnet50_centralized.pth")
print("\n💾 Model saved as resnet50_centralized.pth")


🖥️ Using device: cpu
✅ Dataset loaded from: C:\Users\Admin\OneDrive\Desktop\ResearchTrack\CASIA_FL_Project\data
Total images: 1721
Train: 1290 | Test: 344 | Eval: 87

🚀 Starting centralized training...

📊 Epoch [1/15]
Loss      : 0.6931
Accuracy  : 0.5349
Precision : 0.5452
Recall    : 0.5317
F1 Score  : 0.4956
------------------------------------------------------------

📊 Epoch [2/15]
Loss      : 0.6587
Accuracy  : 0.5465
Precision : 0.5574
Recall    : 0.5437
F1 Score  : 0.5163
------------------------------------------------------------

📊 Epoch [3/15]
Loss      : 0.6355
Accuracy  : 0.5640
Precision : 0.5699
Recall    : 0.5620
F1 Score  : 0.5503
------------------------------------------------------------

📊 Epoch [4/15]
Loss      : 0.6185
Accuracy  : 0.5727
Precision : 0.5746
Recall    : 0.5715
F1 Score  : 0.5677
------------------------------------------------------------

📊 Epoch [5/15]
Loss      : 0.6054
Accuracy  : 0.5610
Precision : 0.5612
Recall    : 0.5603
F1 Score  : 0.5591